# Работа с Excel

## Лабораторная работа 7

In [46]:
import pandas as pd
import openpyxl
import xlwings as xw
from openpyxl import load_workbook
import numpy as np
from xlwings.constants import AutoFillType
import win32api

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [35]:
reviews = pd.read_csv('C:/lb7/reviews_sample.csv')
recipes = pd.read_csv('C:/lb7/recipes_sample_with_tags_ingredients.csv')
recipes = recipes.drop(columns=['contributor_id', 'n_steps', 'ingredients'])
print(reviews, '\n\n', recipes)

             id     user_id  recipe_id        date  rating  \
0        370476       21752      57993  2003-05-01       5   
1        624300      431813     142201  2007-09-16       5   
2        187037      400708     252013  2008-01-10       4   
3        706134  2001852463     404716  2017-12-11       5   
4        312179       95810     129396  2008-03-14       5   
...         ...         ...        ...         ...     ...   
126691  1013457     1270706     335534  2009-05-17       4   
126692   158736     2282344       8701  2012-06-03       0   
126693  1059834      689540     222001  2008-04-08       5   
126694   453285  2000242659     354979  2015-06-02       5   
126695   691207      463435     415599  2010-09-30       5   

                                                   review  
0       Last week whole sides of frozen salmon fillet ...  
1       So simple and so tasty!  I used a yellow capsi...  
2       Very nice breakfast HH, easy to make and yummy...  
3       These a

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [36]:
num_reviews = int(reviews.shape[0])
num_recipes = int(recipes.shape[0])
n_reviews = num_reviews * 0.05
n_recipes = num_recipes * 0.05

reviews_sampels = reviews.sample(n=int(n_reviews))
recipes_sampels = recipes.sample(n=int(n_recipes))

print(reviews_sampels, '\n\n', recipes_sampels)

writer = pd.ExcelWriter('recipes2.xlsx')
reviews_sampels.to_excel(writer, 'Отзывы')
recipes_sampels.to_excel(writer, 'Рецепты')
writer.save()

             id  user_id  recipe_id        date  rating  \
27172   1108730   129958      30358  2006-10-18       5   
31641    681112  1196631     108248  2011-05-12       1   
94398    433060   126440      28648  2005-01-17       5   
50337     34025   765619       7491  2010-12-23       5   
124566   596119   464327     305210  2010-03-24       5   
...         ...      ...        ...         ...     ...   
16068    111947   126440      81347  2009-09-09       4   
14537   1123720   402790      57679  2008-05-14       5   
93613    883752    65803      52282  2005-10-21       5   
122429  1042901   594704      17073  2009-01-08       5   
107546   215595   209747     127262  2005-10-18       5   

                                                   review  
27172   I have been cooking pasta like this for a coup...  
31641   i have no idea what went wrong. i followed the...  
94398   I halved the recipe and cooked it for 8 min. a...  
50337   Delicious! My entire family loved this reci

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [37]:
recipes_list = (recipes_sampels['minutes'] * 60).to_numpy()

print(recipes_list)

wb = xw.Book('recipes2.xlsx')
wb.sheets[1].activate()
wb.sheets[1].range('H2').options(transpose=True).value = recipes_list
wb.sheets[1].range('H1').value = 'seconds_assing'

[29700   300 23700 ...  2700  1800  3600]


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [38]:
wb = xw.Book('recipes2.xlsx')
wb.sheets[1].activate()
wb.sheets[1].range('H2').options(transpose=True).value = recipes_list
wb.sheets[1].range('H1').value = 'seconds_assing'
rng_name = xw.Range('D2').expand('down').address
print(rng_name)
xw.Range('I1').value = 'seconds_formula'
xw.Range('I2').formula = f'=D2 * 60'
xw.Range('I2').options(transpose=True).api.AutoFill(xw.Range('I2:I1501').api, AutoFillType.xlFillDefault)

$D$2:$D$1501


True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
wb = xw.Book('recipes2.xlsx')
sheet = wb.sheets['Рецепты']
xw.Range('J1').value = 'n_reviews'
xw.Range('J2').formula = f'=СЧЁТЕСЛИ(Отзывы!D$2:Отзывы!D$6335;Рецепты!C2)'
xw.Range('J2').options(transpose=True).api.AutoFill(xw.Range('J2:J1501').api, AutoFillType.xlFillDefault)

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [51]:
wb = xw.Book('recipes2.xlsx')
sheet = wb.sheets['Рецепты']
headers_range = sheet.range('h1').expand('right')
headers_range.api.Font.Bold = True
headers_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [54]:
workbook = xw.Book('recipes2.xlsx')
sheet = workbook.sheets['Рецепты']
minutes_range = sheet.range('D2:D1501')
for cell in minutes_range:
    if cell.value < 5:
        cell.color = (0, 255, 0)  # зеленый цвет
    elif 5 <= cell.value < 10:
        cell.color = (255, 255, 0)  # желтый цвет
    else:
        cell.color = (255, 0, 0)  # красный цвет

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    # Открываем книгу
    wb = xw.Book('recipes2.xlsx')
    # Получаем доступ к листам
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']
    # Определяем последнюю строку с данными в листах
    last_row_reviews = reviews_sheet.range('A' + str(reviews_sheet.cells.last_cell.row)).end('up').row
    last_row_recipes = recipes_sheet.range('A' + str(recipes_sheet.cells.last_cell.row)).end('up').row
    # Определяем диапазоны для рейтингов и названий рецептов
    ratings_range = reviews_sheet.range('F2:F' + str(last_row_reviews))
    recipes_range = reviews_sheet.range('D2:D' + str(last_row_reviews))
    # Проходимся по каждой строке из диапазона рейтингов
    for i, rating_cell in enumerate(ratings_range):
        rating = rating_cell.value
        # Проверяем, что рейтинг в диапазоне от 0 до 5
        if rating < 0 or rating > 5:
            reviews_sheet.range('A' + str(i+2)).color = (255, 0, 0)
        else:
            # Проверяем, что соответствующий рецепт имеется на листе Рецепты
            recipe_name = recipes_range[i].value
            recipe_exists = False
            for j in range(2, last_row_recipes+1):
                if recipes_sheet.range('C'+str(j)).value == recipe_name:
                    recipe_exists = True
                    break
            if not recipe_exists:
                reviews_sheet.range('A' + str(i+2)).color = (255, 0, 0)
validate()